<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/KL_Divergence_Ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# カルバック・ライブラー情報量を理解する

https://www.kaggle.com/code/meaninglesslives/understanding-kl-divergence/notebook を DeepL で翻訳したものです。

カルバック・ライブラー情報量とは、ある確率分布（$P$）が2つ目の確率分布（$Q$）とどの程度異なるかを示す尺度です。もし2つの分布が同じなら、カルバック・ライブラー情報量は0になるはずです。したがって、カルバック・ライブラー情報量を最小化することで、$P$に近似する2番目の分布($Q$)のパラメータを見つけることができます。

この投稿では、分布$P$（2つのガウス分布の和）を、もう1つのガウシアン分布$Q$とのカルバック・ライブラー情報量を最小化することで近似してみます。


# ライブラリの読み込み

In [ ]:
import pdb
import numpy as np
import torch
from torch.autograd import grad
import torch.nn.functional as F
import matplotlib.pyplot as plt

# for animation
%matplotlib inline
import matplotlib.animation
from IPython.display import Image

import warnings
warnings.filterwarnings('ignore')

from typing import Dict, List, Tuple

# ガウス分布の構築

Pytorchは特定の種類のディストリビューションからサンプルを取得する簡単な方法を提供します。torch.distributesにはよく使われる分布がたくさんあります。
まず、$$\mu_{1}=-5, \sigma_{1}=1$$と$$\mu_{1}=10, ˶=1$$ の2つのガウス分布を作ってみましょう。

In [ ]:
mu1,sigma1 = -5,1
mu2,sigma2 = 10,1

gaussian1 = torch.distributions.Normal(mu1,sigma1)
gaussian2 = torch.distributions.Normal(mu2,sigma2)

# 健全性チェック
期待されるパラメータを持つガウス分布かどうかを検証するために、いくつかのポイントで分布をサンプリングしてみましょう。

In [ ]:
plt.figure(figsize=(14,4))
x = torch.linspace(mu1-5*sigma1,mu1+5*sigma1,1000)
plt.subplot(1,2,1)
plt.plot(x.numpy(),gaussian1.log_prob(x).exp().numpy())
plt.title(f'$\mu={mu1},\sigma={sigma1}$')

x = torch.linspace(mu2-5*sigma2,mu2+5*sigma2,1000)
plt.subplot(1,2,2)
plt.plot(x.numpy(),gaussian2.log_prob(x).exp().numpy())
plt.title(f'$\mu={mu2},\sigma={sigma2}$')

plt.suptitle('Plotting the distributions')

上の図は、分布が正しく構成されていることを示しています。

ガウシアンを加えて、新しい分布$P(x)$を生成してみましょう。

我々の目的は、この新しい分布を$Q(x)$を使って近似することである。分布$P(x)$と$Q(x)$の間のカルバック・ライブラー情報量を最小化することで、パラメータ $\mu_{Q},\sigma_{Q}$を求めよう。

In [ ]:
plt.figure(figsize=(14,4))
x = torch.linspace(-mu1-mu2-5*sigma1-5*sigma2,mu1+mu2+5*sigma1+5*sigma2,1000)
px = gaussian1.log_prob(x).exp() + gaussian2.log_prob(x).exp()
plt.subplot(1,2,2)
plt.plot(x.numpy(),px.numpy())
plt.title('$P(X)$')

## $Q(X)$ の構成

$P(X)$を近似するためにガウス分布を使う。分布$P(x)$を最もよく近似する最適なパラメータは分かりません。

そこで、単純に$\mu=0,\sigma=1$とします。

$P(x)$を近似しようとしている分布についての予備知識があるので、もっと良い値を選ぶこともできます。しかし、実際のシナリオではそうでないことがほとんどです。

In [ ]:
mu = torch.tensor([0.0])
sigma = torch.tensor([1.0])

plt.figure(figsize=(14,4))
x = torch.linspace(-mu1-mu2-5*sigma1-5*sigma2,mu1+mu2+5*sigma1+5*sigma2,1000)
Q = torch.distributions.Normal(mu,sigma) # this should approximate P, eventually :-)
qx = Q.log_prob(x).exp()
plt.subplot(1,2,2)
plt.plot(x.numpy(),qx.detach().numpy())
plt.title('$Q(X)$')

## KL-Divergence
$$D_{KL}(P(x)||Q(X)) = \sum_{x \in X} P(x) \log(P(x) / Q(x))$$


### pytorch での計算

Pytorchはカルバック・ライブラー情報量を計算する関数を提供しています。詳しくは[こちら](https://pytorch.org/docs/stable/nn.html#torch.nn.functional.kl_div)を参照してください。

注意すべき点は、与えられた入力には対数確率が含まれていることです。ターゲットは確率として（つまり対数を取らずに）与えられます。

したがって、関数の第1引数はQ、第2引数はP（目標分布）となります。

また、数値の安定性にも注意しなければなりません。

In [ ]:
px = gaussian1.log_prob(x).exp() + gaussian2.log_prob(x).exp()
qx = Q.log_prob(x).exp()
F.kl_div(qx.log(),px)

発散は無限大 ^^; この問題は、指数計算をしてから対数計算をしたときに発生すると思います。対数値を直接使うのは問題ないようです。

In [ ]:
px = gaussian1.log_prob(x).exp() + gaussian2.log_prob(x).exp()
qx = Q.log_prob(x)
F.kl_div(qx,px)

In [ ]:
def optimize_loss(px: torch.tensor, loss_fn: str, muq: float = 0.0, sigmaq: float = 1.0,\
                  subsample_factor:int = 3,mode:str = 'min') -> Tuple[float,float,List[int]]:

    mu = torch.tensor([muq],requires_grad=True)
    sigma = torch.tensor([sigmaq],requires_grad=True)

    opt = torch.optim.Adam([mu,sigma])

    loss_val = []
    epochs = 10000

    #required for animation
    all_qx,all_mu = [],[]
    subsample_factor = 3 #have to subsample to reduce memory usage

    torch_loss_fn = getattr(F,loss_fn)
    for i in range(epochs):
        Q = torch.distributions.Normal(mu,sigma) # this should approximate P
        if loss_fn!='kl_div': # we need to exponentiate q(x) for these and few other cases
            qx = Q.log_prob(x).exp()
            all_qx.append(qx.detach().numpy()[::subsample_factor])
        else:
            qx = Q.log_prob(x)
            all_qx.append(qx.exp().detach().numpy()[::subsample_factor])

        if mode=='min':
            loss = torch_loss_fn(qx,px)
        else:
            loss = -torch_loss_fn(qx,px,dim=0)
    #   backward pass
        opt.zero_grad()
        loss.backward()
        opt.step()
        loss_val.append(loss.detach().numpy())
        all_mu.append(mu.data.numpy()[0])


        if i%(epochs//10)==0:
            print('Epoch:',i,'Loss:',loss.data.numpy(),'mu',mu.data.numpy()[0],'sigma',sigma.data.numpy()[0])


    print('Epoch:',i,'Loss:',loss.data.numpy(),'mu',mu.data.numpy()[0],'sigma',sigma.data.numpy()[0])

    plt.figure(figsize=(14,6))
    plt.subplot(2,2,1)
    plt.plot(loss_val)
    plt.xlabel('epoch')
    plt.ylabel(f'{loss_fn} (Loss)')
    plt.title(f'{loss_fn} vs epoch')

    plt.subplot(2,2,2)
    plt.plot(all_mu)
    plt.xlabel('epoch')
    plt.ylabel('$\mu$')
    plt.title('$\mu$ vs epoch')

    return mu.data.numpy()[0],sigma.data.numpy()[0],all_qx

In [ ]:
x = torch.linspace(-mu1-mu2-5*sigma1-5*sigma2,mu1+mu2+5*sigma1+5*sigma2,1000)
px = gaussian1.log_prob(x).exp() + gaussian2.log_prob(x).exp()
mu,sigma,all_qx = optimize_loss(px, loss_fn='kl_div', muq = 0.0, sigmaq = 1.0)

In [ ]:
def create_animation(x:torch.tensor,px:torch.tensor,all_qx:List,subsample_factor:int = 3,\
                     fn:str = 'anim_distr.gif') -> None:

    # create a figure, axis and plot element
    fig = plt.figure()
    ax = plt.axes(xlim=(x.min(),x.max()), ylim=(0,0.5))
    text = ax.text(3,0.3,0)
    line1, = ax.plot([], [], color = "r")
    line2, = ax.plot([], [], color = "g",alpha=0.7)

    def animate(i):
    #     non uniform sampling, interesting stuff happens fast initially
        if i<75:
            line1.set_data(x[::subsample_factor].numpy(),all_qx[i*50])
            text.set_text(f'epoch={i*50}')
            line2.set_data(x[::subsample_factor].numpy(),px.numpy()[::subsample_factor])
        else:
            line1.set_data(x[::subsample_factor].numpy(),all_qx[i*100])
            text.set_text(f'epoch={i*100}')
            line2.set_data(x[::subsample_factor].numpy(),px.numpy()[::subsample_factor])

        return [line1,line2]

    ani = matplotlib.animation.FuncAnimation(fig,animate,frames=100
                                   ,interval=200, blit=True)

    fig.suptitle(f'Minimizing the {fn[:-3]}')
    ax.legend(['Approximation','Actual Distribution'])
    # save the animation as gif
    ani.save(fn, writer='imagemagick', fps=10)

In [ ]:
# %% capture if you dont want to display the final image
ani = create_animation(x,px,all_qx,fn='kl_div.gif')
Image("kl_div.gif")

$P$と$Q$の間の平均二乗距離を解いてみるとどうなるか見てみましょう。

In [ ]:
x = torch.linspace(-mu1-mu2-5*sigma1-5*sigma2,mu1+mu2+5*sigma1+5*sigma2,1000)
px = gaussian1.log_prob(x).exp() + gaussian2.log_prob(x).exp()
mu,sigma,all_qx = optimize_loss(px, loss_fn='mse_loss', muq = 0.0, sigmaq = 1.0)

In [ ]:
fn = 'mse_loss_mean0.gif'
ani = create_animation(x,px,all_qx,fn=fn)
Image(f"{fn}")

結果はカルバック・ライブラー情報量の場合とは大きく異なることがわかります。ガウス分布の1つに向かって収束しており、中間値はありません！

また、$\mu_{Q}$の初期値を変えて実験してみてください。10(2番目のガウス分布の平均)に近い値を選ぶと、それに向かって収束します。

In [ ]:
x = torch.linspace(-mu1-mu2-5*sigma1-5*sigma2,mu1+mu2+5*sigma1+5*sigma2,1000)
px = gaussian1.log_prob(x).exp() + gaussian2.log_prob(x).exp()
mu,sigma,all_qx = optimize_loss(px, loss_fn='mse_loss', muq = 5.0, sigmaq = 1.0)

fn = 'mse_loss_mean5.gif'
ani = create_animation(x,px,all_qx,fn=fn)
Image(f"../working/{fn}")

L1ロスの場合もそうであることは容易に想像がつくでしょう。

では、2つの分布間の余弦類似度を最大化しようとするとどうなるか、試してみましょう。


In [ ]:
x = torch.linspace(-mu1-mu2-5*sigma1-5*sigma2,mu1+mu2+5*sigma1+5*sigma2,1000)
px = gaussian1.log_prob(x).exp() + gaussian2.log_prob(x).exp()
mu,sigma,all_qx = optimize_loss(px, loss_fn='cosine_similarity', muq = 5.0, sigmaq = 1.0,mode='max')

fn = 'cosine_similarity.gif'
ani = create_animation(x,px,all_qx,fn=fn)
Image(f"{fn}")

## 結論
上記のように1次元の場合、最も近い平均値に収束します。複数の谷がある高次元空間では、MSE/L1 Lossを最小化すると異なる結果になる可能性があります。 ディープラーニングでは、ニューラルネットワークの重みをランダムに初期化します。そのため、同じニューラルネットワークの異なる実行において、異なる局所最小値に向かって収束するのは理にかなっています。
確率的重み平均のようなテクニックは、異なる局所最小値への重みを提供するため、おそらく汎化性が向上します。異なる局所最小値がデータセットに関する重要な情報を内包している可能性があります。